In [80]:
import pandas as pd
import numpy as np

In [81]:
df = pd.read_csv('ecom_processed.csv')
df.head()

,Unnamed: 0,user_id,region,device,channel,session_start,session_end,sessiondurationsec,session_date,month,day,hour_of_day,order_dt,revenue,payment_type,promo_code,payer,time_of_day,final_price
0,0,529697267522,United States,iPhone,социальные сети,2019-05-01 00:06:40,2019-05-01 00:07:06,26,2019-05-01,5,3,0,2019-05-01 00:06:40,9999.0,Mobile payments,0.0,1,ночь,9999.0
1,1,601292388085,United States,PC,organic,2019-05-01 06:56:16,2019-05-01 07:09:18,782,2019-05-01,5,3,7,NaN,NaN,NaN,NaN,0,утро,NaN
2,2,852898876338,United States,Mac,социальные сети,2019-05-01 04:30:45,2019-05-01 04:34:56,251,2019-05-01,5,3,4,NaN,NaN,NaN,NaN,0,ночь,NaN
3,3,998513020664,United States,iPhone,социальные сети,2019-05-01 18:53:42,2019-05-01 18:57:35,233,2019-05-01,5,3,18,NaN,NaN,NaN,NaN,0,вечер,NaN
4,4,240702200943,United States,Mac,социальные сети,2019-05-02 14:04:32,2019-05-02 14:09:51,319,2019-05-02,5,4,14,NaN,NaN,NaN,NaN,0,день,NaN


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1008 non-null   int64  
 1   user_id             1008 non-null   int64  
 2   region              1008 non-null   object 
 3   device              1008 non-null   object 
 4   channel             1008 non-null   object 
 5   session_start       1008 non-null   object 
 6   session_end         1008 non-null   object 
 7   sessiondurationsec  1008 non-null   int64  
 8   session_date        1008 non-null   object 
 9   month               1008 non-null   int64  
 10  day                 1008 non-null   int64  
 11  hour_of_day         1008 non-null   int64  
 12  order_dt            281 non-null    object 
 13  revenue             281 non-null    float64
 14  payment_type        281 non-null    object 
 15  promo_code          281 non-null    float64
 16  payer 

Средний чек

In [83]:
df.final_price.mean().round(2)

5403.63

Среднее кол-во покупок от пользователя

In [84]:
# vc = df.user_id.value_counts()
# for id in df['user_id']:
#     if vc[id] == 2:
#         look = df[df['user_id'] == id]

df.payer.mean().round(2)

0.28

Средняя продолжительность сессии по каналам рекламы

In [85]:
df.groupby('channel').sessiondurationsec.mean().round(2)

channel
email-рассылки         1982.12
organic                1750.33
контекстная реклама    1872.46
реклама у блогеров     1679.98
социальные сети        1607.04
Name: sessiondurationsec, dtype: float64

Средняя продолжительность сессии по типу устройства

In [86]:
df.groupby('device').sessiondurationsec.mean().round(2)

device
Android    1815.64
Mac        1660.60
PC         1801.67
iPhone     1629.69
Name: sessiondurationsec, dtype: float64

Топ-3 рекламных канала по среднему чеку

In [87]:
df.groupby('channel').final_price.mean().sort_values(ascending=False).round(2).head(3)

channel
социальные сети       5515.96
organic               5428.55
реклама у блогеров    5388.66
Name: final_price, dtype: float64

Топ-3 региона по среднему чеку

In [88]:
df.groupby('region').final_price.mean().sort_values(ascending=False).round(2).head(3)

region
UK               5522.53
United States    5416.35
Germany          5331.00
Name: final_price, dtype: float64

Топ-3 месяца по среднему чеку с разбивкой по регионам

In [89]:
def func_month(x):
    if x == 5:
        return "May"
    if x == 6:
        return "June"
    if x == 7:
        return "July"
    if x == 8:
        return "August"
    if x == 9:
        return "September"
    if x == 10:
        return "October"


df['month'] = pd.DatetimeIndex(df['session_date']).month
df['month'] = df['month'].apply(func_month)

In [90]:
reg_df = pd.DataFrame()
reg_df['region'] = ['United States'] * 6 + ['France'] * 6 + ['Germany'] * 6 + ['UK'] * 6
reg_df['month'] = ['May', 'June', 'July', 'August', 'September', 'October'] * 4
reg_df['mean_price'] = list(df.groupby(['region', 'month']).final_price.mean().round(2))
reg_df

,region,month,mean_price
0,United States,May,4999.00
1,United States,June,4999.00
2,United States,July,5499.00
3,United States,August,5449.00
4,United States,September,5165.67
5,United States,October,4874.00
6,France,May,4987.89
7,France,June,7499.00
8,France,July,5679.00
9,France,August,4665.67


In [91]:
reg_df.groupby(['region', 'month']).agg({'mean_price':sum})['mean_price'].groupby('region', group_keys=False).apply(lambda x : x.sort_values(ascending=False).head(3))

C:\Users\Димончелло\AppData\Local\Temp\ipykernel_22012\2883707028.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  reg_df.groupby(['region', 'month']).agg({'mean_price':sum})['mean_price'].groupby('region', group_keys=False).apply(lambda x : x.sort_values(ascending=False).head(3))


region         month    
France         June         7499.00
               July         5679.00
               October      5249.00
Germany        September    6056.14
               August       5999.00
               May          5499.00
UK             August       6051.17
               May          5555.25
               July         5459.00
United States  July         5499.00
               August       5449.00
               September    5165.67
Name: mean_price, dtype: float64

MAU* по каждому месяцу с разбивкой по рекламным каналам\
*MAU (Monthly Active Users) — это количество уникальных пользователей за месяц.

In [92]:
df.groupby(['month', 'channel']).agg({'user_id':pd.Series.nunique})

user_id
month     channel                     
August    email-рассылки             4
          organic                   61
          контекстная реклама       37
          реклама у блогеров        20
          социальные сети           68
July      email-рассылки             3
          organic                   46
          контекстная реклама       24
          реклама у блогеров        16
          социальные сети           57
June      email-рассылки             1
          organic                   59
          контекстная реклама       28
          реклама у блогеров        13
          социальные сети           53
May       email-рассылки             7
          organic                   76
          контекстная реклама        8
          реклама у блогеров        14
          социальные сети           41
October   email-рассылки             6
          organic                   47
          контекстная реклама       27
          реклама у блогеров        18
          социальные сети           85
September email-рассылки             3
          organic                   58
          контекстная реклама       38
          реклама у блогеров        20
          социальные сети           67

In [93]:
chan_df = pd.DataFrame()
chan_df['channel'] = df['channel'].unique()
chan_df['users'] = chan_df['channel'].apply(lambda x: df[df['channel'] == x].shape[0])
chan_df['unique_users'] = chan_df['channel'].apply(lambda x: len(df[df['channel'] == x].user_id.unique()))
chan_df['paying_users'] = chan_df['channel'].apply(lambda x: df[df['channel'] == x].payer.sum())
chan_df['overall_revenue'] = chan_df['channel'].apply(lambda x: df[df['channel'] == x].revenue.sum())
chan_df.sort_values(by='overall_revenue', ascending=False)

,channel,users,unique_users,paying_users,overall_revenue
0,социальные сети,374,369,112,634888.0
1,organic,347,346,88,492912.0
3,контекстная реклама,162,159,45,238955.0
2,реклама у блогеров,101,100,29,157971.0
4,email-рассылки,24,23,7,37993.0
